In [8]:
import csv
from datetime import datetime, timedelta

# Constants
total_classes = 1094
sessions_per_class = {
    "4-session": 120,
    "2-session": 636,
    "1-session": 338,
}
instructor_count = 20  # Increase the number of instructors to meet the constraints
days_per_week = 7
starting_date = datetime(2024, 1, 1)  # Starting date for the schedule
us_holidays = [datetime(2024, 1, 1), datetime(2024, 5, 27), datetime(2024, 7, 4), datetime(2024, 9, 2), datetime(2024, 12, 25)]

# Initialize instructor schedules
instructor_schedules = {i: [] for i in range(1, instructor_count + 1)}

# Days of the week to avoid (Saturday and Sunday)
days_to_avoid = [5, 6]

# Generate the CSV file
with open('2024_instructor_schedule.csv', 'w', newline='') as csvfile:
    fieldnames = ['Instructor', 'Class ID', 'Session Type', 'Date', 'Day of the Week']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()

    class_id = 1
    for session_type, count in sessions_per_class.items():
        for _ in range(count):
            for instructor in range(1, instructor_count + 1):
                # Find the next available day for the instructor, avoiding Saturday, Sunday, and holidays
                next_day = len(instructor_schedules[instructor]) % days_per_week
                while next_day in days_to_avoid or starting_date + timedelta(days=next_day) in us_holidays:
                    next_day = (next_day + 1) % days_per_week

                # If it's the first session of a class, update the class ID
                if next_day == 0:
                    class_id += 1

                session_date = starting_date + timedelta(days=next_day)
                session_id = (class_id - 1) * 4 + 1  # Calculate the session ID within the class

                # Add the session to the schedule
                instructor_schedules[instructor].append(class_id)

                # Write the session to the CSV
                writer.writerow({
                    'Instructor': instructor,
                    'Class ID': class_id,
                    'Session Type': session_type,
                    'Date': session_date.strftime('%Y-%m-%d'),
                    'Day of the Week': session_date.strftime('%A'),
                })

                next_day = (next_day + 1) % days_per_week

print("Calendar generated and saved as '2024_instructor_schedule.csv'.")



Calendar generated and saved as '2024_instructor_schedule.csv'.
